<a href="https://colab.research.google.com/github/MiPf0/cameraExercise/blob/master/TransformingDisasterTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
#from https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX or https://mccormickml.com/2019/07/22/BERT-fine-tuning/
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
#from https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX or https://mccormickml.com/2019/07/22/BERT-fine-tuning/
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [3]:
#from https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX or https://mccormickml.com/2019/07/22/BERT-fine-tuning/
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.7 MB/s eta 0:00:00


## Import Training Dataset

In [8]:
#from https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX or https://mccormickml.com/2019/07/22/BERT-fine-tuning/
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("train_merged.tsv", delimiter='\t')

# Report the number of sentences.
print('Number of tweets: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of tweets: 53,531



,event_name_label,event_type_label,tweet_id,tweet_text,class_label
36746,hurricane_maria,hurricane,910777423876231168,Indiana Task Force 1 will leave from Wright-Pa...,other_relevant_information
6121,canada_wildfires,wildfires,729643536229011456,"Hey @KenIlgunas, would love to hear your thoug...",not_humanitarian
25072,hurricane_harvey,hurricane,902857594175463424,The death toll for #Harvey has risen to at lea...,injured_or_dead_people
48018,kerala_floods,floods,1030401452538089477,"Northies, Did u know the entire Kerala was eff...",infrastructure_and_utility_damage
40859,italy_earthquake_aug,earthquake,768506897104269314,Donate Now #ItalyEarthquake Disaster Relief #T...,rescue_volunteering_or_donation_effort
48518,kerala_floods,floods,1031165354658033664,there are nearly 50 people stranded in thrissu...,requests_or_urgent_needs
15214,hurricane_dorian,hurricane,1168233151438761984,"A completely incompetent President, while the ...",not_humanitarian
49335,midwestern_us_floods,floods,1112893610171789312,RT @weatherdak: Incredible before &amp; after ...,other_relevant_information
36181,hurricane_maria,hurricane,914207500269948928,Puerto Rico is Facing A Severe Humanitarian Cr...,other_relevant_information
29200,hurricane_irma,hurricane,908139918043570182,"@mbachelet @antonioguterres WTF, This bitch ki...",not_humanitarian


## Class Label Encoding

In [9]:
#from https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX or https://mccormickml.com/2019/07/22/BERT-fine-tuning/
df.loc[df.class_label == 'injured_or_dead_people'].sample(5)[['tweet_text', 'class_label']]

,tweet_text,class_label
44384,#Leptospirosis claims another life in flood-af...,injured_or_dead_people
2661,#CampFire: 5 more deaths for a total of 76. In...,injured_or_dead_people
6895,"Cyclone Idai: 1 000 Killed In Mozambique, 217 ...",injured_or_dead_people
10388,Derry nun among those killed in Ecuador earthq...,injured_or_dead_people
23646,"With 6 deaths in a population of 7 million, hu...",injured_or_dead_people


In [10]:
#from https://github.com/donglinchen/text_classification/blob/master/feature_extraction.ipynb
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder().fit(df["class_label"])
df['encoded_class'] = le.transform(df['class_label'])

In [12]:
#self
df.sample(20)

,event_name_label,event_type_label,tweet_id,tweet_text,class_label,encoded_class
26908,hurricane_harvey,hurricane,901405220466503680,#HurricaneHarvey now a Cat 1 storm. Catastroph...,caution_and_advice,0
34675,hurricane_maria,hurricane,914463292718043137,@sanrous2 Teamsters are trying to rally 100 vo...,rescue_volunteering_or_donation_effort,8
15152,hurricane_dorian,hurricane,1167514839163572227,@NOLAjewelry504 I heard on CNN and MSNBC today...,displaced_people_and_evacuations,1
25448,hurricane_harvey,hurricane,902654920284921857,Insurance Won’t Cover Damage To 80% Of Homes F...,infrastructure_and_utility_damage,2
19017,hurricane_florence,hurricane,1039561407023509505,#FlorenceSC #evacuation #shelter locations wil...,displaced_people_and_evacuations,1
47111,kerala_floods,floods,1033354102430883843,Is it true that Bollywood actor Shah Rukh Khan...,rescue_volunteering_or_donation_effort,8
22233,hurricane_harvey,hurricane,901693058865090560,"Harvey continues to weaken inland, but catastr...",other_relevant_information,6
27259,hurricane_harvey,hurricane,903681057274384386,Buses filled with #Harvey evacuees to be airli...,displaced_people_and_evacuations,1
22617,hurricane_harvey,hurricane,902869571283165184,"UPDATE: #Harvey to make landfall again, Texas ...",injured_or_dead_people,3
45512,kerala_floods,floods,1030391678706884608,ὄDwe Heartly Salute u Indian ARMY I appeal to ...,rescue_volunteering_or_donation_effort,8


In [13]:
# from https://github.com/dblilienthal/Multiclass-Text-Classification-with-DistilBERT-on-COVID-19-Tweets/blob/main/Multiclass%20Text%20Classification%20with%20DistilBERT%20on%20COVID-19%20Tweets.ipynb
label_dict = (df[['class_label','encoded_class']].drop_duplicates()
              .sort_values(by='encoded_class')
              .reset_index(drop=True)['class_label']
              .to_dict())

for index, key in label_dict.items():
    print(index, key)

0 caution_and_advice
1 displaced_people_and_evacuations
2 infrastructure_and_utility_damage
3 injured_or_dead_people
4 missing_or_found_people
5 not_humanitarian
6 other_relevant_information
7 requests_or_urgent_needs
8 rescue_volunteering_or_donation_effort
9 sympathy_and_support
